# Decoding a substitution cipher using the Metropolis-Hastings algorithm 

This notebook describes how we can use the Metropolis-Hastings algorithm to break a simple substitution cipher. It expands on the story described in Perci Diaconis' 2009 paper titled [The Markov Chain Monte Carlo Revolution](https://www.ams.org/journals/bull/2009-46-02/S0273-0979-08-01238-X/S0273-0979-08-01238-X.pdf). 

The story in the introduction of the paper goes that the California state prison psychologist approached the Stanford statistics department to help them out with a problem: Prisoners were exchanging coded messages and they wanted to decode the messages. The folks at Stanford conjectured that the prisoners were systematically mapping characters to others using a fixed key---that is, they were using a substitution cipher---and used the Metropolis-Hastings algorithm to find the key.

## Substitution ciphers

Substitution ciphers encode and decode plain text using a key that maps each character to another character. Suppose the key was the character map in the table below:


|original | encoded |
|---------|---------|
|a        | z       |
|p        | 4       |
|l        | 5       |
|e        | a       |

Then the word `apple` should be mapped to `z445a`. 

Let us implement the encoding function.

In [1]:
def code(text, key): 
    return ''.join(list(map(lambda char: key[char], text)))

In [2]:
code('apple', {'a': 'z', 'p': '4', 'l': '5', 'e': 'a'})

'z445a'

Decoding the cipher text can be performed by applying the reverse mapping.

In [3]:
code('z445a', {'z': 'a', '4': 'p', '5': 'l', 'a': 'e'})

'apple'

## Scoring keys

If we want to decode some encrypted text, we need to find a key that works. In other words, we need to search over the space of keys. Note that the key is a discrete object here -- it is a permutation of the alphabet. 

To guide this search, we need some way to quantify what makes one key better than another. Intuitively, a good key is one that makes the decoded text readable. In order to score readability of some text, we can use as a surrogate the probability of the text looking like English. As a coarse approximation of this probability, we will use character-level bigram probabilities.

For any sequence of characters $c_1c_2c_3\cdots c_n$, we will compute its probability (that is, the probability of it being English) as

$$P(c_1c_2c_3\cdots c_n) = P(c_1) P(c_2 \mid c_1) P(c_3 \mid c_2) \cdots P(c_n \mid c_{n-1})$$

From the above example, the probability of `apple` is $P(a)P(p \mid a)P(p \mid p)P(l \mid p)P(e \mid l)$.

To make things simple, we will ignore the first character because it is not going to make much of a difference anyway. Note that to calculate this probability, we will need a table of bigram transition probabilities, i.e. $P(c | c')$, for every pair of characters $c$ and $c'$. If we have this bigram probability table, then we can score sequences of characters. 

**Implementation notes**: 

1. To avoid underflow, we will only keep log probabilities, not probabilities. So all the multiplications will become additions. That is, we are actually computing
$$\log P(c_1c_2c_3\cdots c_n) = \log P(c_1) + \log P(c_2 \mid c_1) + \cdots  + \log P(c_n \mid c_{n-1})$$

2. If our bigram log probabilities table does not have a particular character pair, then we need to allocate a small probability (or a small log probability) to this rare event.

In [4]:
LOG_EPSILON = -20

def goodness(text, bigram_table):
    bigrams = zip(text, text[1:])
    sum = 0
    for b in bigrams:
        sum += bigram_table.get(b) or LOG_EPSILON
    return sum

## Calculating the bigram log probabilities

We still need to take care of one more things before we can get to the Metropolis-Hastings part. We need a way to get those bigram log probabilities that are used to score how good a key is. 

For simplicity, suppose we only deal with upper case alphanumeric characters and spaces.

In [5]:
alphabet = "ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789 "

With this alphabet of 37 characters, we need transition probablities for all pairs of characters. That is we need $37^2 = 1369$ probabilities (actually, $37\cdot 36 = 1332$ numbers because every $P(\cdot | c)$ should sum to 1). In any case, this is a rather small number of probabilities to compute. We can obtain reasonably reliable estimates of these probabilities from a large collection of text. 

We need to read the text from some file and clean it up by keeping only those characters that are in our alphabet.

In [6]:
import re

multispace_regex = re.compile('\s+')
alphanumeric_regex = re.compile('[^A-Z0-9 ]+')

def cleanup(string):
    upper = string.upper()
    only_alphanumeric = alphanumeric_regex.sub(' ', upper).strip()
    single_spaced = multispace_regex.sub(' ', only_alphanumeric)
    return single_spaced

def read_file(file):
    with open(file, 'r') as f:
        return cleanup(f.read())

At this point, we have everything we need to calculate the bigram log probabilities.

In [7]:
from collections import Counter
import math

def bigram_log_probabilities(file):
    text = read_file(file)
    
    character_counts = Counter(text)
    bigram_counts = Counter(zip(text, text[1:]))
    bigrams = bigram_counts.keys()
    
    return dict(map(lambda k: [k, math.log(bigram_counts[k]) - math.log(character_counts[k[0]])], bigrams))

We can calucate the log probabilities from any text. If the text is sufficiently large, we will get good enough estimates. Let us use *War and peace*. It has 3076470 characters in all, which should be good enough.

In [8]:
book = 'data/war-and-peace.txt'

bigram_table = bigram_log_probabilities(book)

Let us see what the `bigram_table` has. We can ask for log probabilities of pairs of characters.

In [9]:
print([bigram_table[('W', 'A')], bigram_table[('W', 'U')]])

[-1.579580457348568, -7.6167927784159115]


And we can use the goodness function above to get log probabilities of text. Keep in mind that this log probability is estimated *according* to this bigram model.

In [10]:
print("Log probability of PEACE = " +str(goodness("PEACE", bigram_table)))

Log probability of PEACE = -9.744572103997422


In [11]:
print("Log probability of PEACW= " +str(goodness("PEACW", bigram_table)))

Log probability of PEACW= -17.27482215251294


Unsurprisingly, `PEACE` has a higher score than `PEACW`.

## Breaking the substitution cipher

To break the cipher, we will start off with a random key (i.e. a random permutation of the alphabet) and iteratively improve upon it. Let us make a random key first.

In [12]:
import random

def make_random_key():
    l = list(alphabet)
    random.shuffle(l)
    return dict(zip(alphabet, l))

This will create a random key that looks like an expanded version of the table above.

At the heart of the decoding algorithm is the idea that we will navigate the space of keys. This means that from any key, we need to be able to move to a random neighbor. We can define neighborhood very simply by transposing the mappings for two randomly chosen characters.

In [13]:
def transpose_random(key):
    pair = random.sample(alphabet, 2)
    new_key = dict(key)
    new_key[pair[0]] = key[pair[1]]
    new_key[pair[1]] = key[pair[0]]
    return new_key

Now we can implement the Metropolis algorithm. We will start off with a randomly constructed key and iterate for thousands of steps. At each step, the algorithm will decode the key to produce a candidate plain text. Remember that if the key is good, the plain text should be readable English. In other words, we can score the key using the bigram model we have.

Our goal is to improve the score as the iterations progress. To do that, we can move to a random neighbor of the current key by transposing the mappings of two randomly chosen characters and then compute the new key's score. If the new score is better than the old one, we will immediately move to the proposed key. 

If the new key is not better, we will still allow for the possiblity of moving. To do that, we can toss a coin whose bias is the ratio of the two probabilities. (This number should be between zero and one because the new key has a lower score.) The coin decides whether we will move to the proposed key. 

The important thing to note here is that the search can sometimes move to a worse state than where it is, but moving to a really bad state is a low probability event. 

In [14]:
def decode(cipher_text, bigram_table, iters = 10000, print_every = 1000):
    current_key = make_random_key()
    
    scores = []
    
    for i in range(0, iters):
        decoded = code(cipher_text, current_key)
        score = goodness(decoded, bigram_table)
        
        if i % print_every == 0:
            print(str(i) + "\t" + decoded + "\n")
        
        changed_key = transpose_random(current_key)
        changed_score = goodness(code(cipher_text, changed_key), bigram_table)

        if changed_score > score:
            current_key = changed_key
        else:
            diff = changed_score - score
            if math.log(random.random())  < diff:
                current_key = changed_key

    decoded = code(cipher_text, current_key)
    print("Final decoded: " + decoded)
    return decoded

Let us take this for a spin. To start off, let us create a new key for encoding text.

In [15]:
key = make_random_key()

print(key)

{'A': 'H', 'B': '7', 'C': 'C', 'D': 'W', 'E': '3', 'F': 'K', 'G': 'U', 'H': 'V', 'I': 'M', 'J': 'Q', 'K': '8', 'L': '9', 'M': 'D', 'N': '6', 'O': 'J', 'P': 'E', 'Q': 'Y', 'R': 'I', 'S': 'F', 'T': 'A', 'U': 'P', 'V': 'L', 'W': '4', 'X': ' ', 'Y': 'S', 'Z': '5', '0': 'G', '1': 'T', '2': '1', '3': 'Z', '4': 'R', '5': 'N', '6': '2', '7': 'X', '8': '0', '9': 'B', ' ': 'O'}


Let us also read some plain text. The file `data/plaintext.txt` has the first few lines from the book *The Wallpaper*.

In [16]:
plain_text = read_file('data/plaintext.txt')

print(plain_text)

IT IS VERY SELDOM THAT MERE ORDINARY PEOPLE LIKE JOHN AND MYSELF SECURE ANCESTRAL HALLS FOR THE SUMMER A COLONIAL MANSION A HEREDITARY ESTATE I WOULD SAY A HAUNTED HOUSE AND REACH THE HEIGHT OF ROMANTIC FELICITY BUT THAT WOULD BE ASKING TOO MUCH OF FATE STILL I WILL PROUDLY DECLARE THAT THERE IS SOMETHING QUEER ABOUT IT


We can now encode the plain text using the randomly generated key.

In [17]:
cipher_text = code(plain_text, key)

print(cipher_text)

MAOMFOL3ISOF39WJDOAVHAOD3I3OJIWM6HISOE3JE93O9M83OQJV6OH6WODSF39KOF3CPI3OH6C3FAIH9OVH99FOKJIOAV3OFPDD3IOHOCJ9J6MH9ODH6FMJ6OHOV3I3WMAHISO3FAHA3OMO4JP9WOFHSOHOVHP6A3WOVJPF3OH6WOI3HCVOAV3OV3MUVAOJKOIJDH6AMCOK39MCMASO7PAOAVHAO4JP9WO73OHF8M6UOAJJODPCVOJKOKHA3OFAM99OMO4M99OEIJPW9SOW3C9HI3OAVHAOAV3I3OMFOFJD3AVM6UOYP33IOH7JPAOMA


This looks nothing like the plain text. Hopefully the Metropolis algorithm will decode it.

In [18]:
decoded = decode(cipher_text, bigram_table, 10000)

0	QEXQ5XG2H0X52TRKWXEIYEXW2H2XKHRQAYH0XF2KFT2XTQB2X4KIAXYARXW052TMX528LH2XYA825EHYTXIYTT5XMKHXEI2X5LWW2HXYX8KTKAQYTXWYA5QKAXYXI2H2RQEYH0X25EYE2XQXVKLTRX5Y0XYXIYLAE2RXIKL52XYARXH2Y8IXEI2XI2Q7IEXKMXHKWYAEQ8XM2TQ8QE0XULEXEIYEXVKLTRXU2XY5BQA7XEKKXWL8IXKMXMYE2X5EQTTXQXVQTTXFHKLRT0XR28TYH2XEIYEXEI2H2XQ5X5KW2EIQA7XOL22HXYUKLEXQE

1000	IP IS JERY SENMAT PLOP TERE ARMIKORY GEAGNE NIVE ZALK OKM TYSEND SECURE OKCESPRON LONNS DAR PLE SUTTER O CANAKION TOKSIAK O LEREMIPORY ESPOPE I WAUNM SOY O LOUKPEM LAUSE OKM REOCL PLE LEIHLP AD RATOKPIC DENICIPY BUP PLOP WAUNM BE OSVIKH PAA TUCL AD DOPE SPINN I WINN GRAUMNY MECNORE PLOP PLERE IS SATEPLIKH FUEER OBAUP IP

2000	IT IS ZEMY SELFOC TRAT CEME OMFINAMY PEOPLE LIKE WORN ANF CYSELV SEDUME ANDESTMAL RALLS VOM TRE SUCCEM A DOLONIAL CANSION A REMEFITAMY ESTATE I HOULF SAY A RAUNTEF ROUSE ANF MEADR TRE REIGRT OV MOCANTID VELIDITY BUT TRAT HOULF BE ASKING TOO CUDR OV VATE STILL I HILL PMOUFLY FEDLAME TRAT TREME IS SOCETRING QUEEM ABOUT IT

3000	IT IS VERY SEL

To visually compare the original plaintext and the decoded version, we can diff them and inspect the differences. Let's write a helper function that can highlight the differences.

In [19]:
from IPython.display import display, HTML

def compare(original_plaintext, decoded):
    output = ['<p>']
    insert = []
    delete = []
    num_differences = 0
    for o, d in zip(original_plaintext, decoded):
        if o == d:
            output.extend(insert)
            output.extend(delete)
            insert = []
            delete = []
            output.append(o)
        else:
            insert.append(f'<span style="color:green">{d}</span>')
            delete.append(f'<del style="color:red">{o}</del>')
            num_differences = num_differences + 1
    output.append('</p>')
    
    display(HTML(''.join(output)))
    display(HTML(f'<p><i>{num_differences} characters different.'))
    


In [20]:
compare(plain_text, decoded)

Let's look at a different example, about the 2014 Japanese Grand Prix. Many of the words in this text were meaningless when *War and Peace* was written. Would the decoding still work?

In [21]:
plain_text = cleanup(
    """The 2014 Japanese Grand Prix was a Formula One motor race held on 5 October 2014 
    at the Suzuka Circuit in Suzuka, Mie. It was the 15th race of the 2014 FIA Formula 
    One World Championship, and the 30th Japanese Grand Prix of the Formula One era. 
    The 44-lap race was won by Mercedes driver Lewis Hamilton, increasing his lead in the 
    World Drivers' Championship to ten points over his teammate, Nico Rosberg, who 
    finished second. Red Bull Racing driver Sebastian Vettel came in third. Heavy rain 
    from Typhoon Phanfone soaked the track surface and reduced visibility. Jules Bianchi
    lost control of his Marussia on the 43rd lap and collided with a tractor crane that 
    was tending to Adrian Sutil's car, which had spun off on the previous lap. Bianchi 
    sustained severe head injuries and died nine months later, the first death of a driver
    in a Formula One Grand Prix since Ayrton Senna's in 1994. """
)

In [22]:
cipher_text = code(plain_text, key)

decoded = decode(cipher_text, bigram_table, 20000)

0	LI7J9MNPJ AUA1767JT0A1QJU0EOJBA6JAJFV0GZSAJV17JGVLV0J0AC7JI7SQJV1JXJVCLVD70J9MNPJALJLI7J6Z8ZHAJCE0CZELJE1J6Z8ZHAJGE7JELJBA6JLI7JNXLIJ0AC7JVFJLI7J9MNPJFEAJFV0GZSAJV17JBV0SQJCIAGUEV16IEUJA1QJLI7JKMLIJ AUA1767JT0A1QJU0EOJVFJLI7JFV0GZSAJV17J70AJLI7JPPJSAUJ0AC7JBA6JBV1JDWJG70C7Q76JQ0E470JS7BE6JIAGESLV1JE1C07A6E1TJIE6JS7AQJE1JLI7JBV0SQJQ0E4706JCIAGUEV16IEUJLVJL71JUVE1L6JV470JIE6JL7AGGAL7J1ECVJ0V6D70TJBIVJFE1E6I7QJ67CV1QJ07QJDZSSJ0ACE1TJQ0E470J67DA6LEA1J47LL7SJCAG7JE1JLIE0QJI7A4WJ0AE1JF0VGJLWUIVV1JUIA1FV17J6VAH7QJLI7JL0ACHJ6Z0FAC7JA1QJ07QZC7QJ4E6EDESELWJ ZS76JDEA1CIEJSV6LJCV1L0VSJVFJIE6JGA0Z66EAJV1JLI7JPK0QJSAUJA1QJCVSSEQ7QJBELIJAJL0ACLV0JC0A17JLIALJBA6JL71QE1TJLVJAQ0EA1J6ZLESJ6JCA0JBIECIJIAQJ6UZ1JVFFJV1JLI7JU074EVZ6JSAUJDEA1CIEJ6Z6LAE17QJ674707JI7AQJE1 Z0E76JA1QJQE7QJ1E17JGV1LI6JSAL70JLI7JFE06LJQ7ALIJVFJAJQ0E470JE1JAJFV0GZSAJV17JT0A1QJU0EOJ6E1C7JAW0LV1J6711AJ6JE1JN55P

1000	RYE 1302 GALANEDE FTANS LTIZ WAD A PUTCOMA UNE CURUT TABE YEMS UN 7 UBRUHET 1302 AR RYE DOXOJA BITBOIR IN DOXOJA CIE 

10000	THE 7812 JAPANESE GRAND PRIX WAS A FORMULA ONE MOTOR RACE HELD ON 3 OCTOBER 7812 AT THE SUZUKA CIRCUIT IN SUZUKA MIE IT WAS THE 13TH RACE OF THE 7812 FIA FORMULA ONE WORLD CHAMPIONSHIP AND THE 08TH JAPANESE GRAND PRIX OF THE FORMULA ONE ERA THE 22 LAP RACE WAS WON BY MERCEDES DRIVER LEWIS HAMILTON INCREASING HIS LEAD IN THE WORLD DRIVERS CHAMPIONSHIP TO TEN POINTS OVER HIS TEAMMATE NICO ROSBERG WHO FINISHED SECOND RED BULL RACING DRIVER SEBASTIAN VETTEL CAME IN THIRD HEAVY RAIN FROM TYPHOON PHANFONE SOAKED THE TRACK SURFACE AND REDUCED VISIBILITY JULES BIANCHI LOST CONTROL OF HIS MARUSSIA ON THE 20RD LAP AND COLLIDED WITH A TRACTOR CRANE THAT WAS TENDING TO ADRIAN SUTIL S CAR WHICH HAD SPUN OFF ON THE PREVIOUS LAP BIANCHI SUSTAINED SEVERE HEAD INJURIES AND DIED NINE MONTHS LATER THE FIRST DEATH OF A DRIVER IN A FORMULA ONE GRAND PRIX SINCE AYRTON SENNA S IN 1662

11000	THE 7812 JAPANESE GRAND PRIX WAS A FORMULA ONE MOTOR RACE HELD ON 3 OCTOBER 7812 AT THE SUZUKA CIRCUIT IN SUZUKA

Final decoded: THE 1802 JAPANESE GRAND PRIX WAS A FORMULA ONE MOTOR RACE HELD ON 7 OCTOBER 1802 AT THE SUZUKA CIRCUIT IN SUZUKA MIE IT WAS THE 07TH RACE OF THE 1802 FIA FORMULA ONE WORLD CHAMPIONSHIP AND THE 38TH JAPANESE GRAND PRIX OF THE FORMULA ONE ERA THE 22 LAP RACE WAS WON BY MERCEDES DRIVER LEWIS HAMILTON INCREASING HIS LEAD IN THE WORLD DRIVERS CHAMPIONSHIP TO TEN POINTS OVER HIS TEAMMATE NICO ROSBERG WHO FINISHED SECOND RED BULL RACING DRIVER SEBASTIAN VETTEL CAME IN THIRD HEAVY RAIN FROM TYPHOON PHANFONE SOAKED THE TRACK SURFACE AND REDUCED VISIBILITY JULES BIANCHI LOST CONTROL OF HIS MARUSSIA ON THE 23RD LAP AND COLLIDED WITH A TRACTOR CRANE THAT WAS TENDING TO ADRIAN SUTIL S CAR WHICH HAD SPUN OFF ON THE PREVIOUS LAP BIANCHI SUSTAINED SEVERE HEAD INJURIES AND DIED NINE MONTHS LATER THE FIRST DEATH OF A DRIVER IN A FORMULA ONE GRAND PRIX SINCE AYRTON SENNA S IN 0662


In [23]:
compare(plain_text, decoded)

Different runs of the decoder produces different mappings, but most of the times, it seems that the decoder does not get numbers correct. (Exercise: Why is this a reasonable type of mistake given our assumptions?)